In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import os
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import lightgbm as lgb
os.chdir('/home/sambeet/data/dsg/')

In [2]:
trade = pd.read_csv('trade_recfeats.csv')
test = pd.read_csv('test_recfeats.csv')

In [3]:
market=pd.read_csv('Market.csv',parse_dates=['DateKey'])
market['YearKey']=market.DateKey.dt.year
market['YearKey_Prev']=market.DateKey.dt.year
market['WeekKey']=market.DateKey.dt.weekofyear
market['WeekKey_Prev_1']=market['WeekKey']+1
market['WeekKey_Prev_2']=market['WeekKey']+2
market['WeekKey_Prev_3']=market['WeekKey']+3
market[market['WeekKey_Prev_1']==54].WeekKey_Prev_1=1
market[market['WeekKey_Prev_1']==54].YearKey_Prev=market[market['WeekKey_Prev_1']==0].YearKey_Prev+1
market[market['WeekKey_Prev_2']==54].WeekKey_Prev_2=1
market[market['WeekKey_Prev_2']==54].YearKey_Prev=market[market['WeekKey_Prev_2']==0].YearKey_Prev-1
market[market['WeekKey_Prev_2']==55].WeekKey_Prev_2=2
market[market['WeekKey_Prev_2']==55].YearKey_Prev=market[market['WeekKey_Prev_2']==55].YearKey_Prev+1
market[market['WeekKey_Prev_3']==54].WeekKey_Prev_3=1
market[market['WeekKey_Prev_3']==54].YearKey_Prev=market[market['WeekKey_Prev_3']==54].YearKey_Prev+1
market[market['WeekKey_Prev_3']==55].WeekKey_Prev_3=2
market[market['WeekKey_Prev_3']==55].YearKey_Prev=market[market['WeekKey_Prev_3']==55].YearKey_Prev+1
market[market['WeekKey_Prev_3']==56].WeekKey_Prev_3=3
market[market['WeekKey_Prev_3']==56].YearKey_Prev=market[market['WeekKey_Prev_3']==56].YearKey_Prev+1

aggregations_2 = {
    'Price':['mean'],
    'ZSpread':['mean'],
    'Yield':['mean']
}
WeekDataGroup=market.groupby(['IsinIdx','YearKey','WeekKey']).agg(aggregations_2)
WeekDataGroup.columns = ["_".join(x) for x in WeekDataGroup.columns.ravel()]
WeekDataGroup.reset_index(level=0, inplace=True)
WeekDataGroup.reset_index(level=0, inplace=True)
WeekDataGroup.reset_index(level=0, inplace=True)
WeekDataGroup_1=market.groupby(['IsinIdx','YearKey_Prev','WeekKey_Prev_1']).agg(aggregations_2)
WeekDataGroup_1.columns = ["_".join(x) for x in WeekDataGroup_1.columns.ravel()]
WeekDataGroup_1.reset_index(level=0, inplace=True)
WeekDataGroup_1.reset_index(level=0, inplace=True)
WeekDataGroup_1.reset_index(level=0, inplace=True)
WeekDataGroup_2=market.groupby(['IsinIdx','YearKey_Prev','WeekKey_Prev_2']).agg(aggregations_2)
WeekDataGroup_2.columns = ["_".join(x) for x in WeekDataGroup_2.columns.ravel()]
WeekDataGroup_2.reset_index(level=0, inplace=True)
WeekDataGroup_2.reset_index(level=0, inplace=True)
WeekDataGroup_2.reset_index(level=0, inplace=True)
WeekDataGroup_3=market.groupby(['IsinIdx','YearKey_Prev','WeekKey_Prev_3']).agg(aggregations_2)
WeekDataGroup_3.columns = ["_".join(x) for x in WeekDataGroup_3.columns.ravel()]
WeekDataGroup_3.reset_index(level=0, inplace=True)
WeekDataGroup_3.reset_index(level=0, inplace=True)
WeekDataGroup_3.reset_index(level=0, inplace=True)
WeekDataGroup_1.rename({'ZSpread_mean':'ZSpread_mean_lag1','Price_mean':'Price_mean_lag1','Yield_mean':'Yield_mean_lag1'},axis=1,inplace=True)
WeekDataGroup_2.rename({'ZSpread_mean':'ZSpread_mean_lag2','Price_mean':'Price_mean_lag2','Yield_mean':'Yield_mean_lag2'},axis=1,inplace=True)
WeekDataGroup_3.rename({'ZSpread_mean':'ZSpread_mean_lag3','Price_mean':'Price_mean_lag3','Yield_mean':'Yield_mean_lag3'},axis=1,inplace=True)
trade=trade.merge(WeekDataGroup_1,how='left',left_on=['IsinIdx','year','weekofyear'],right_on=['IsinIdx','YearKey_Prev','WeekKey_Prev_1'])
trade=trade.merge(WeekDataGroup_2,how='left',left_on=['IsinIdx','year','weekofyear'],right_on=['IsinIdx','YearKey_Prev','WeekKey_Prev_2'])
trade=trade.merge(WeekDataGroup_3,how='left',left_on=['IsinIdx','year','weekofyear'],right_on=['IsinIdx','YearKey_Prev','WeekKey_Prev_3'])
trade.drop(['YearKey_Prev','YearKey_Prev_x','YearKey_Prev_y','WeekKey_Prev_1','WeekKey_Prev_2','WeekKey_Prev_3'], axis=1,inplace=True)
test=test.merge(WeekDataGroup_1,how='left',left_on=['IsinIdx','year','weekofyear'],right_on=['IsinIdx','YearKey_Prev','WeekKey_Prev_1'])
test=test.merge(WeekDataGroup_2,how='left',left_on=['IsinIdx','year','weekofyear'],right_on=['IsinIdx','YearKey_Prev','WeekKey_Prev_2'])
test=test.merge(WeekDataGroup_3,how='left',left_on=['IsinIdx','year','weekofyear'],right_on=['IsinIdx','YearKey_Prev','WeekKey_Prev_3'])
test.drop(['YearKey_Prev','YearKey_Prev_x','YearKey_Prev_y','WeekKey_Prev_1','WeekKey_Prev_2','WeekKey_Prev_3'], axis=1,inplace=True)

/home/sambeet/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [7]:
trade['ZSpread_mean_lag1'].fillna(0,inplace=True)
trade['ZSpread_mean_lag2'].fillna(0,inplace=True)
trade['ZSpread_mean_lag3'].fillna(0,inplace=True)
trade['Price_mean_lag1'].fillna(0,inplace=True)
trade['Price_mean_lag2'].fillna(0,inplace=True)
trade['Price_mean_lag3'].fillna(0,inplace=True)
trade['Yield_mean_lag1'].fillna(0,inplace=True)
trade['Yield_mean_lag2'].fillna(0,inplace=True)
trade['Yield_mean_lag3'].fillna(0,inplace=True)
test['ZSpread_mean_lag1'].fillna(0,inplace=True)
test['ZSpread_mean_lag2'].fillna(0,inplace=True)
test['ZSpread_mean_lag3'].fillna(0,inplace=True)
test['Price_mean_lag1'].fillna(0,inplace=True)
test['Price_mean_lag2'].fillna(0,inplace=True)
test['Price_mean_lag3'].fillna(0,inplace=True)
test['Yield_mean_lag1'].fillna(0,inplace=True)
test['Yield_mean_lag2'].fillna(0,inplace=True)
test['Yield_mean_lag3'].fillna(0,inplace=True)
trade.head()

,CustomerIdx,IsinIdx,BuySell,year,weekofyear,CustomerInterest,predCustomerInterest,ZSpread_mean_lag1,Price_mean_lag1,Yield_mean_lag1,ZSpread_mean_lag2,Price_mean_lag2,Yield_mean_lag2,ZSpread_mean_lag3,Price_mean_lag3,Yield_mean_lag3
0,0,16471,0,2016,1,False,0.329668,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0,16471,0,2016,6,False,0.330347,4.7658,106.4876,5.5628,4.8914,105.6000,5.9872,4.6570,106.5500,5.5630
2,0,16471,0,2016,20,False,0.330303,3.8594,106.9748,4.6424,4.0806,106.7748,4.8794,4.1390,106.7198,4.9742
3,0,16471,0,2016,27,False,0.329193,2.8292,107.6250,3.5210,2.8286,107.6376,3.5982,2.9100,107.6374,3.6784
4,0,16471,0,2016,46,False,0.330132,2.9484,105.9254,3.9278,3.0168,105.9750,3.9830,2.8418,106.1300,3.8312


In [8]:
customer = pd.read_csv('Modified_Customer_Features.csv')
customer.drop(['Unnamed: 0','NotionalEUR_sum','NotionalEUR_sum','Price_sum','TradeDateKey_amin','TradeDateKey_amax',
               'Interested_Count','Interested_Percentage','BondMaturingDays_sum','BondAgeDays_sum',
               'BondDuration_sum','IssuedAmount_sum','Sector','Subsector','Country'],axis=1,inplace=True)
customer = customer.rename({'Region':'CustomerRegion'},axis=1)
categorical_columns = ['CustomerRegion']
for col in categorical_columns:
    customer[col] = customer[col].astype('category')
    print 'Transforming {}...'.format(col)
    encoder = LabelEncoder()
    encoder.fit(customer[col].astype(str))
    customer[col] = encoder.transform(customer[col].astype(str))
bond = pd.read_csv('Isin.csv',parse_dates=['ActualMaturityDateKey', 'IssueDateKey'])
bond = bond.rename({'Region':'BondRegion'},axis=1)
categorical_columns = ['Seniority','Currency','ActivityGroup','BondRegion','Activity',
       'RiskCaptain','Owner','CompositeRating','IndustrySector','IndustrySubgroup','MarketIssue','CouponType']
for col in categorical_columns:
    bond[col] = bond[col].astype('category')
    print 'Transforming {}...'.format(col)
    encoder = LabelEncoder()
    encoder.fit(bond[col].astype(str))
    bond[col] = encoder.transform(bond[col].astype(str))
bond['MaturityPeriod'] = ((bond['ActualMaturityDateKey'] - bond['IssueDateKey'])/365.).dt.days
bond['IssuedAmount'] = np.log1p(bond.IssuedAmount/1000000)
bond.drop(['TickerIdx','ActualMaturityDateKey','IssueDateKey','Seniority','Currency','ActivityGroup','BondRegion','Activity','RiskCaptain','Owner','CompositeRating','IndustrySector','IndustrySubgroup','MarketIssue','CouponType'],axis=1,inplace=True)
customer['Tenure'] =  customer['Tenure'].apply(lambda x: x.split(' ')[0])
customer['Vintage'] =  customer['Vintage'].apply(lambda x: x.split(' ')[0])
customer['Tenure'] = customer['Tenure'].astype(int)
customer['Vintage'] = customer['Vintage'].astype(int)
customer['Sell_Transactions'].fillna(0,inplace=True)

Transforming CustomerRegion...
Transforming Seniority...
Transforming Currency...
Transforming ActivityGroup...
Transforming BondRegion...
Transforming Activity...
Transforming RiskCaptain...
Transforming Owner...
Transforming CompositeRating...
Transforming IndustrySector...
Transforming IndustrySubgroup...
Transforming MarketIssue...
Transforming CouponType...


In [9]:
int_type = ['CustomerIdx','IsinIdx','BuySell','year','weekofyear','CustomerRegion','IsinIdx_count','MaturityPeriod',
            'IsinIdx_nunique','Buy_Transactions','Sell_Transactions','Tenure','Vintage','Rating_0_Count',
            'Rating_1_Count','Rating_2_Count','Rating_3_Count','Rating_4_Count','CouponType_Step_Count',
            'CouponType_Floating_Count','CouponType_Zero_Count','CouponType_Variable_Count','CouponType_None_Count']
for col in list(trade.columns):
    print col
    if col in int_type:
        trade[col] = trade[col].astype(int)
        trade[col] = pd.to_numeric(trade[col],downcast='integer')
    else:
        trade[col] = pd.to_numeric(trade[col],downcast='float')
for col in list(customer.columns):
    print col
    if col in int_type:
        customer[col] = customer[col].astype(int)
        customer[col] = pd.to_numeric(customer[col],downcast='integer')
    else:
        customer[col] = pd.to_numeric(customer[col],downcast='float')
for col in list(bond.columns):
    print col
    if col in int_type:
        bond[col] = bond[col].astype(int)
        bond[col] = pd.to_numeric(bond[col],downcast='integer')
    else:
        bond[col] = pd.to_numeric(bond[col],downcast='float')
trade.info()

CustomerIdx
IsinIdx
BuySell
year
weekofyear
CustomerInterest
predCustomerInterest
ZSpread_mean_lag1
Price_mean_lag1
Yield_mean_lag1
ZSpread_mean_lag2
Price_mean_lag2
Yield_mean_lag2
ZSpread_mean_lag3
Price_mean_lag3
Yield_mean_lag3
CustomerIdx
IsinIdx_count
IsinIdx_nunique
NotionalEUR_mean
Price_mean
Buy_Transactions
Sell_Transactions
Tenure
Vintage
BondMaturingDays_mean
BondAgeDays_mean
BondDuration_mean
IssuedAmount_mean
Rating_0_Count
Rating_1_Count
Rating_2_Count
Rating_3_Count
Rating_4_Count
CouponType_Step_Count
CouponType_Floating_Count
CouponType_Zero_Count
CouponType_Variable_Count
CouponType_None_Count
CustomerRegion
IsinIdx
IssuedAmount
MaturityPeriod
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30171186 entries, 0 to 30171185
Data columns (total 16 columns):
CustomerIdx             int16
IsinIdx                 int16
BuySell                 int8
year                    int16
weekofyear              int8
CustomerInterest        bool
predCustomerInterest    float32
ZSpre

In [13]:
del WeekDataGroup,WeekDataGroup_1,WeekDataGroup_2,WeekDataGroup_3
gc.collect()

56

In [16]:
bond.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27411 entries, 0 to 27410
Data columns (total 3 columns):
IsinIdx           27411 non-null int16
IssuedAmount      27411 non-null float32
MaturityPeriod    27411 non-null int16
dtypes: float32(1), int16(2)
memory usage: 214.2 KB


In [17]:
test['predCustomerInterest'] = (test.CustomerInterest_1 + test.CustomerInterest_2 + test.CustomerInterest_3 + test.CustomerInterest_4 + test.CustomerInterest_5)/5.
trade = trade.merge(customer,on=['CustomerIdx'],how='left')
trade = trade.merge(bond,on=['IsinIdx'],how='left')
test = test.merge(customer,on=['CustomerIdx'],how='left')
test = test.merge(bond,on=['IsinIdx'],how='left')
gc.collect()

14

In [23]:
trade.to_feather('trade_custmod_bond_market.feather')

In [24]:
test.to_feather('test_custmod_bond_market.feather')

In [1]:
import pyarrow.feather as pyfa
x = pyfa.read_feather('trade_custmod_bond_market.feather',nthreads=8)